In [1]:
import pandas as pd
import numpy as np

import catboost as cb
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
df = pd.read_csv('train_contest.csv', sep='✺')
df.head()

#df['Цена'] = df['Цена'].str.replace(' ', '')
#df['Цена'] = df['Цена'].str.replace('руб.', '')
#df['Цена'] = df["Цена"].astype(float)
#print(df[(df['Количество_комнат'] != 'студия')]['Цена'].mean())
#print(df[(df['Количество_комнат'] == 'студия')]['Цена'].mean())
#print(df[(df['Количество_комнат'] == '1')]['Цена'].mean())
#print(df[(df['Количество_комнат'] == '2')]['Цена'].mean())
#print(df[(df['Количество_комнат'] == '3')]['Цена'].mean())

/tmp/ipykernel_50154/3848673271.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('train_contest.csv', sep='✺')


,Город,Район,Улица,Номер_дома,Тип_сделки,Срок_аренды,Цена,Тип_дома,Количество_комнат,Общая_площадь,Количество_этажей,Материал_стен,Тип_продажи,Фото,Описание,Дата,Телефон,Площадь_кухни,Жилая_площадь,Этаж
0,ул. Глухарская,Не указано,Не указано,60.03534;30.21263,Продажа,Не аренда,5 216 640,Новостройка,1,33,Не указано,монолитный,Не указано,https://98.img.avito.st/image/1/F36L7rayu5e9R3...,1-комнатная 33 кв.м с предчистовой отделкой на...,2021-03-24,https://avito.ru/sankt-peterburg/kvartiry/1-k....,Не указано,Не указано,22 из 24
1,Пискаревский пр-т,Не указано,Не указано,60.00606;30.45463,Продажа,Не аренда,8 829 528,Новостройка,3,68.3,Не указано,монолитный,Не указано,https://87.img.avito.st/image/1/8o12m7ayXmRAMp...,В продаже 3-комнатная квартира с отделкой в но...,2021-04-10,https://avito.ru/sankt-peterburg/kvartiry/3-k....,14.8,38.8,10 из 13
2,Санкт-Петербург,Приморский район,Не указано,60.000162082374;30.306065460388,Продажа,Не аренда,9 760 000,Вторичка,2,64.6,Не указано,кирпичный,Не указано,https://23.img.avito.st/image/1/VmEtyray-ogbYz...,Продам 2-ую квартиру в 300-ста метрах от метро...,2021-04-20,https://avito.ru/sankt-peterburg/kvartiry/2-k....,13.3,32.6,12 из 17
3,Санкт-Петербург,Не указано,Дегтярный пер.,20,Продажа,Не аренда,5 900 000,Вторичка,студия,34,Не указано,кирпичный,Не указано,https://70.img.avito.st/image/1/u5RmKbayF31QgN...,"Продажа студии 34м2 на 1этаже, Дегтярный 20, в...",2021-03-03,https://avito.ru/sankt-peterburg/kvartiry/apar...,5,27,1 из 5
4,Санкт-Петербург,Не указано,Соляной пер.,59.945581;30.340635,Продажа,Не аренда,11 000 000,Вторичка,3,64.1,Не указано,кирпичный,Не указано,https://52.img.avito.st/image/1/jbtz-bayIVJFUO...,Эксклюзивное предложение !Продам 3 комнатную к...,too old,https://avito.ru/sankt-peterburg/kvartiry/3-k....,6.3,43.1,4 из 5


In [3]:
import math
def level_(narr):
    lev =[]
    for i in narr:
        if i == None:
            lev.append(None)
        elif type(i) == type(5.0):
            if math.isnan(i):
                lev.append(None)
            else:
                lev.append(int(i))
            
            
        elif len(i.split(' из '))==2:
            l1 , l2 = map(int,i.split(' из '))
            lev.append(l1)
        else:
            lev.append(int(i))
            
    return lev


def preprocess(filename, is_test=False):
    df = pd.read_csv(filename, sep='✺')
    
    df.drop(['Улица', 'Номер_дома', 'Тип_сделки', 'Срок_аренды', 'Описание', 'Телефон', 'Фото', 'Количество_этажей', 'Тип_продажи'],axis=1,inplace=True)
    
    df['lev'] = np.array(level_(df['Этаж'].to_numpy()))
    
    df.drop(['Этаж'],axis=1,inplace=True)
    
    df['data_too_old'] = df['Дата'].apply(lambda x: 1 if x == "too old" else 0)
    
    df.drop(['Дата'],axis=1,inplace=True)
    
    if not is_test:
        df['Цена'] = df['Цена'].str.replace(' ', '')
        df['Цена'] = df['Цена'].str.replace('руб.', '')
        df['Цена'] = df["Цена"].astype(float)

    
    if not is_test:
        X = df.drop(['Цена'],axis=1)
    else:
        X = df
    X['Жилая_площадь'] = X['Жилая_площадь'].str.replace(' м2', '')
    X['Площадь_кухни'] = X['Площадь_кухни'].str.replace(' м2', '')
    X['Общая_площадь'] = X['Общая_площадь'].str.replace(' м2', '')

    X['Площадь_кухни'] = X['Площадь_кухни'].replace("Не указано", np.nan)
    X['Площадь_кухни'] = X["Площадь_кухни"].astype(float)
    X['Жилая_площадь'] = X['Жилая_площадь'].replace("Не указано", np.nan)
    X['Жилая_площадь'] = X["Жилая_площадь"].astype(float)
    X['Общая_площадь'] = X['Общая_площадь'].replace("Не указано", np.nan)
    X['Общая_площадь'] = X["Общая_площадь"].astype(float)
    
    if not is_test:
        Y = df['Цена']
    else:
        Y = None
    
    return X, Y

In [4]:
X, Y = preprocess('train_contest.csv')
print(len(X))
print(len(Y))

/tmp/ipykernel_50154/350630780.py:24: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, sep='✺')


13705
13705


In [5]:
from sklearn.model_selection import train_test_split
x_train,xv,y_train,yv = train_test_split(X,Y,test_size=0.15)

In [6]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

X

,Город,Район,Тип_дома,Количество_комнат,Общая_площадь,Материал_стен,Площадь_кухни,Жилая_площадь,lev,data_too_old
0,ул. Глухарская,Не указано,Новостройка,1,33.0,монолитный,NaN,NaN,22,0
1,Пискаревский пр-т,Не указано,Новостройка,3,68.3,монолитный,14.8,38.8,10,0
2,Санкт-Петербург,Приморский район,Вторичка,2,64.6,кирпичный,13.3,32.6,12,0
3,Санкт-Петербург,Не указано,Вторичка,студия,34.0,кирпичный,5.0,27.0,1,0
4,Санкт-Петербург,Не указано,Вторичка,3,64.1,кирпичный,6.3,43.1,4,1
...,...,...,...,...,...,...,...,...,...,...
13700,Санкт-Петербург,Не указано,Вторичка,студия,21.2,монолитный,NaN,15.1,21,1
13701,Санкт-Петербург,Не указано,Вторичка,3,85.0,панельный,13.0,49.0,16,1
13702,Санкт-Петербург,Пушкинский р-н,Вторичка,2,51.0,монолитный,NaN,NaN,8,0
13703,пос. Парголово,Не указано,Новостройка,1,33.0,монолитный,8.8,16.0,7,0


In [ ]:
model = CatBoostRegressor(
    iterations=55000,
    learning_rate=0.005,
    loss_function='RMSE',
    eval_metric='RMSE',
)

model.fit(
    X, Y,
    cat_features=['Город', 'Тип_дома', 'Материал_стен', 'Район', 'Количество_комнат'],
    verbose=True,
    plot=True
)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 3663773.7937722	total: 50.7ms	remaining: 46m 26s
1:	learn: 3653171.7868299	total: 55.5ms	remaining: 25m 25s
2:	learn: 3642275.9534273	total: 59.3ms	remaining: 18m 7s
3:	learn: 3631261.4216918	total: 62.6ms	remaining: 14m 20s
4:	learn: 3620460.6863522	total: 66.1ms	remaining: 12m 7s
5:	learn: 3609610.3941937	total: 69.2ms	remaining: 10m 33s
6:	learn: 3598950.6368659	total: 72.7ms	remaining: 9m 30s
7:	learn: 3588094.9469173	total: 76ms	remaining: 8m 42s
8:	learn: 3577441.1558656	total: 79.2ms	remaining: 8m 3s
9:	learn: 3567220.2026100	total: 81.9ms	remaining: 7m 30s
10:	learn: 3557054.6658383	total: 84.9ms	remaining: 7m 4s
11:	learn: 3547001.7666472	total: 87.9ms	remaining: 6m 42s
12:	learn: 3536694.6526157	total: 90.7ms	remaining: 6m 23s
13:	learn: 3526688.0859472	total: 94ms	remaining: 6m 9s
14:	learn: 3516454.5669814	total: 96.6ms	remaining: 5m 54s
15:	learn: 3506459.1422285	total: 99ms	remaining: 5m 40s
16:	learn: 3496951.1463881	total: 102ms	remaining: 5m 29s
17:	learn: 34

198:	learn: 2285087.4198377	total: 644ms	remaining: 2m 57s
199:	learn: 2281372.0574405	total: 648ms	remaining: 2m 57s
200:	learn: 2277522.2897965	total: 650ms	remaining: 2m 57s
201:	learn: 2273687.1227772	total: 654ms	remaining: 2m 57s
202:	learn: 2269855.6582607	total: 656ms	remaining: 2m 57s
203:	learn: 2266171.2152123	total: 659ms	remaining: 2m 57s
204:	learn: 2262285.1462958	total: 663ms	remaining: 2m 57s
205:	learn: 2258520.2759131	total: 666ms	remaining: 2m 57s
206:	learn: 2254630.0175695	total: 669ms	remaining: 2m 56s
207:	learn: 2250865.3091557	total: 672ms	remaining: 2m 56s
208:	learn: 2247033.7205411	total: 675ms	remaining: 2m 56s
209:	learn: 2243588.5373682	total: 678ms	remaining: 2m 57s
210:	learn: 2239910.8269374	total: 682ms	remaining: 2m 57s
211:	learn: 2236336.5390736	total: 685ms	remaining: 2m 56s
212:	learn: 2232792.7016345	total: 687ms	remaining: 2m 56s
213:	learn: 2229229.9733570	total: 691ms	remaining: 2m 56s
214:	learn: 2225576.2084910	total: 695ms	remaining: 2m 5

398:	learn: 1846445.6746842	total: 1.23s	remaining: 2m 48s
399:	learn: 1845372.9520911	total: 1.24s	remaining: 2m 48s
400:	learn: 1844272.5728156	total: 1.24s	remaining: 2m 48s
401:	learn: 1843213.0947239	total: 1.24s	remaining: 2m 48s
402:	learn: 1842222.6984752	total: 1.25s	remaining: 2m 48s
403:	learn: 1841210.4959569	total: 1.25s	remaining: 2m 48s
404:	learn: 1840105.9351151	total: 1.25s	remaining: 2m 48s
405:	learn: 1839037.3887968	total: 1.25s	remaining: 2m 48s
406:	learn: 1838046.5297964	total: 1.26s	remaining: 2m 48s
407:	learn: 1837022.8429620	total: 1.26s	remaining: 2m 48s
408:	learn: 1836051.9618585	total: 1.26s	remaining: 2m 48s
409:	learn: 1835046.1490869	total: 1.27s	remaining: 2m 48s
410:	learn: 1834061.4982013	total: 1.27s	remaining: 2m 48s
411:	learn: 1833312.4725865	total: 1.27s	remaining: 2m 48s
412:	learn: 1832271.9905792	total: 1.27s	remaining: 2m 48s
413:	learn: 1831156.8755862	total: 1.28s	remaining: 2m 48s
414:	learn: 1830229.0667852	total: 1.28s	remaining: 2m 4

595:	learn: 1718312.5023951	total: 1.83s	remaining: 2m 46s
596:	learn: 1717987.4515108	total: 1.83s	remaining: 2m 46s
597:	learn: 1717619.0163322	total: 1.83s	remaining: 2m 46s
598:	learn: 1717350.2517593	total: 1.83s	remaining: 2m 46s
599:	learn: 1716980.3314614	total: 1.84s	remaining: 2m 46s
600:	learn: 1716650.3549350	total: 1.84s	remaining: 2m 46s
601:	learn: 1716303.2041350	total: 1.84s	remaining: 2m 46s
602:	learn: 1715931.0335799	total: 1.85s	remaining: 2m 46s
603:	learn: 1715536.2107035	total: 1.85s	remaining: 2m 46s
604:	learn: 1715132.9013256	total: 1.85s	remaining: 2m 46s
605:	learn: 1714719.2712299	total: 1.86s	remaining: 2m 46s
606:	learn: 1714347.1935621	total: 1.86s	remaining: 2m 46s
607:	learn: 1713987.1436357	total: 1.86s	remaining: 2m 46s
608:	learn: 1713603.3708488	total: 1.86s	remaining: 2m 46s
609:	learn: 1713252.9410129	total: 1.87s	remaining: 2m 46s
610:	learn: 1712899.6976848	total: 1.87s	remaining: 2m 46s
611:	learn: 1712440.1060340	total: 1.87s	remaining: 2m 4

...

29803:	learn: 1046068.7404659	total: 2m 18s	remaining: 1m 57s
29804:	learn: 1046057.9096997	total: 2m 18s	remaining: 1m 57s
29805:	learn: 1046057.3002368	total: 2m 18s	remaining: 1m 57s
29806:	learn: 1046055.2984446	total: 2m 18s	remaining: 1m 57s
29807:	learn: 1046037.5696623	total: 2m 18s	remaining: 1m 57s
29808:	learn: 1046034.2098171	total: 2m 18s	remaining: 1m 57s
29809:	learn: 1046032.0397392	total: 2m 18s	remaining: 1m 57s
29810:	learn: 1046030.3092654	total: 2m 18s	remaining: 1m 57s
29811:	learn: 1046015.0390516	total: 2m 18s	remaining: 1m 57s
29812:	learn: 1045992.8481701	total: 2m 18s	remaining: 1m 57s
29813:	learn: 1045989.2026217	total: 2m 18s	remaining: 1m 57s
29814:	learn: 1045983.0664223	total: 2m 18s	remaining: 1m 57s
29815:	learn: 1045979.0002325	total: 2m 18s	remaining: 1m 56s
29816:	learn: 1045978.4948309	total: 2m 18s	remaining: 1m 56s
29817:	learn: 1045978.0549953	total: 2m 18s	remaining: 1m 56s
29818:	learn: 1045971.4403642	total: 2m 18s	remaining: 1m 56s
29819:	l

29975:	learn: 1044761.4075442	total: 2m 19s	remaining: 1m 56s
29976:	learn: 1044758.5868564	total: 2m 19s	remaining: 1m 56s
29977:	learn: 1044756.7287237	total: 2m 19s	remaining: 1m 56s
29978:	learn: 1044751.1243537	total: 2m 19s	remaining: 1m 56s
29979:	learn: 1044746.8895845	total: 2m 19s	remaining: 1m 56s
29980:	learn: 1044730.8189535	total: 2m 19s	remaining: 1m 56s
29981:	learn: 1044724.6428514	total: 2m 19s	remaining: 1m 56s
29982:	learn: 1044718.7027506	total: 2m 19s	remaining: 1m 56s
29983:	learn: 1044709.0819396	total: 2m 19s	remaining: 1m 56s
29984:	learn: 1044704.1135582	total: 2m 19s	remaining: 1m 56s
29985:	learn: 1044702.8771020	total: 2m 19s	remaining: 1m 56s
29986:	learn: 1044693.1334522	total: 2m 19s	remaining: 1m 56s
29987:	learn: 1044673.3491217	total: 2m 19s	remaining: 1m 56s
29988:	learn: 1044665.6376360	total: 2m 19s	remaining: 1m 56s
29989:	learn: 1044655.1758822	total: 2m 19s	remaining: 1m 56s
29990:	learn: 1044651.1420283	total: 2m 19s	remaining: 1m 56s
29991:	l

30147:	learn: 1043493.5890728	total: 2m 20s	remaining: 1m 55s
30148:	learn: 1043490.9962405	total: 2m 20s	remaining: 1m 55s
30149:	learn: 1043486.4761791	total: 2m 20s	remaining: 1m 55s
30150:	learn: 1043478.8952931	total: 2m 20s	remaining: 1m 55s
30151:	learn: 1043474.6057658	total: 2m 20s	remaining: 1m 55s
30152:	learn: 1043470.4662054	total: 2m 20s	remaining: 1m 55s
30153:	learn: 1043461.3271641	total: 2m 20s	remaining: 1m 55s
30154:	learn: 1043455.0709969	total: 2m 20s	remaining: 1m 55s
30155:	learn: 1043449.7472344	total: 2m 20s	remaining: 1m 55s
30156:	learn: 1043442.5329367	total: 2m 20s	remaining: 1m 55s
30157:	learn: 1043434.2839106	total: 2m 20s	remaining: 1m 55s
30158:	learn: 1043412.9837766	total: 2m 20s	remaining: 1m 55s
30159:	learn: 1043400.2103547	total: 2m 20s	remaining: 1m 55s
30160:	learn: 1043382.1532861	total: 2m 20s	remaining: 1m 55s
30161:	learn: 1043370.5978990	total: 2m 20s	remaining: 1m 55s
30162:	learn: 1043367.6485288	total: 2m 20s	remaining: 1m 55s
30163:	l

30294:	learn: 1042382.6338290	total: 2m 21s	remaining: 1m 55s
30295:	learn: 1042376.0268835	total: 2m 21s	remaining: 1m 55s
30296:	learn: 1042363.9425717	total: 2m 21s	remaining: 1m 55s
30297:	learn: 1042360.7371835	total: 2m 21s	remaining: 1m 55s
30298:	learn: 1042355.9355869	total: 2m 21s	remaining: 1m 55s
30299:	learn: 1042354.7186390	total: 2m 21s	remaining: 1m 55s
30300:	learn: 1042345.2306220	total: 2m 21s	remaining: 1m 55s
30301:	learn: 1042342.6283121	total: 2m 21s	remaining: 1m 55s
30302:	learn: 1042336.3490726	total: 2m 21s	remaining: 1m 55s
30303:	learn: 1042320.2504760	total: 2m 21s	remaining: 1m 55s
30304:	learn: 1042314.0767960	total: 2m 21s	remaining: 1m 55s
30305:	learn: 1042308.7030135	total: 2m 21s	remaining: 1m 55s
30306:	learn: 1042307.9865647	total: 2m 21s	remaining: 1m 55s
30307:	learn: 1042303.8848567	total: 2m 21s	remaining: 1m 55s
30308:	learn: 1042288.5450795	total: 2m 21s	remaining: 1m 55s
30309:	learn: 1042287.0846387	total: 2m 21s	remaining: 1m 55s
30310:	l

30430:	learn: 1041403.1898760	total: 2m 22s	remaining: 1m 54s
30431:	learn: 1041392.6059728	total: 2m 22s	remaining: 1m 54s
30432:	learn: 1041386.1018495	total: 2m 22s	remaining: 1m 54s
30433:	learn: 1041380.7253402	total: 2m 22s	remaining: 1m 54s
30434:	learn: 1041375.4075306	total: 2m 22s	remaining: 1m 54s
30435:	learn: 1041375.3266024	total: 2m 22s	remaining: 1m 54s
30436:	learn: 1041372.1693611	total: 2m 22s	remaining: 1m 54s
30437:	learn: 1041367.6638558	total: 2m 22s	remaining: 1m 54s
30438:	learn: 1041361.0878442	total: 2m 22s	remaining: 1m 54s
30439:	learn: 1041357.9438063	total: 2m 22s	remaining: 1m 54s
30440:	learn: 1041352.8492521	total: 2m 22s	remaining: 1m 54s
30441:	learn: 1041348.5147751	total: 2m 22s	remaining: 1m 54s
30442:	learn: 1041343.7216040	total: 2m 22s	remaining: 1m 54s
30443:	learn: 1041315.9048456	total: 2m 22s	remaining: 1m 54s
30444:	learn: 1041313.4546983	total: 2m 22s	remaining: 1m 54s
30445:	learn: 1041306.2044998	total: 2m 22s	remaining: 1m 54s
30446:	l

In [ ]:
# parameters = {'depth': [4, 6, 10], 
#               'learning_rate' : [0.02, 0.04, 0.06], 
#               'iterations'    : [200, 300, 400]}

# fit_params = {}

# model = CatBoostRegressor()

# Grid_CBC = GridSearchCV(estimator=model, param_grid = parameters, cv = 4, n_jobs=-1)
# Grid_CBC.fit(X, Y, cat_features= ['Город', 'Тип_дома', 'Материал_стен'])

# Grid_CBC.best_params_


In [ ]:
X_test, ignore = preprocess('for_prediction.csv', is_test=True)

ts = pd.read_csv('for_prediction.csv', sep='✺')
X_test = X_test.drop(['Id'],axis=1)

sub = model.predict(X_test)
sub_df = pd.DataFrame()
sub_df['Id'] = ts['Id']
sub_df['Predicted'] = sub
sub_df.to_csv('sub.csv',index=False)